In [ ]:
### Project name: a comparison of accuracy of 4 pre-trained models

# Explanation:
# a series of optional experiments for a comparison of accuracy of 4 pre-trained models:
# vgg16, ResNet50, EfficientNetB7 and InceptionV3
# all based on dataset of Cat and Dog Images (kaggle)
# optionally adopting StratifiedKFold for learning improvement
# adopting ReduceLROnPlateau: Reduce learning rate when a metric has stopped improving. Models 
# often benefit from reducing the learning rate by a factor of 2-10 once learning stagnates. 
# it is an on-going project, not well organized


# experiment data gathered are as following:
# basic setting: vgg16 + ReduceLROnPlateau + optimizer=Adam + 10 epochs
# Epoch 10
# 625/625 [==============================] - 974s 2s/step - loss: 0.2726 - accuracy: 0.8808 - val_loss: 0.2409 - val_accuracy: 0.8990

# vgg16 load_model, fit again -> best result
# Epoch 4/10
# 625/625 [==============================] - 1084s 2s/step - loss: 0.2741 - accuracy: 0.8787 - val_loss: 0.2292 - val_accuracy: 0.9010
# Epoch 10/10
# 625/625 [==============================] - 1013s 2s/step - loss: 0.2580 - accuracy: 0.8874 - val_loss: 0.2263 - val_accuracy: 0.9002

# vgg16 being replaced with ResNet50 
# Epoch 10/10
# 625/625 [==============================] - 679s 1s/step - loss: 0.6296 - accuracy: 0.6448 - val_loss: 0.6026 - val_accuracy: 0.6821

# basic setting replaced by calling CyclicLR to adjust learning rate, optimizer=torch.optim.RMSprop 
# Epoch 10/10
# 625/625 [==============================] - 976s 2s/step - loss: 0.2878 - accuracy: 0.8723 - val_loss: 0.2656 - val_accuracy: 0.8892

# vgg16 being replaced with EfficientNetB7 - val_accuracy: 0.5016

# InceptionV3 + ReduceLROnPlateau + optimizer=Adam + StratifiedKFold
# Epoch 7/10
# 625/625 [==============================] - 831s 1s/step - loss: 0.1707 - accuracy: 0.9280 - val_loss: 0.1345 - val_accuracy: 0.9495

# vgg16 + ReduceLROnPlateau + optimizer=Adam + StratifiedKFold - val_accuracy: 0.8868

In [ ]:
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os
import keras

#check if GPU is available
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

#Define Image properties
images_weight = 128
images_height = 128
image_size = (images_weight, images_height)
image_channels=3

#Prepare dataset for training model
import re

filenames = os.listdir("C:\\Users\\xxx\\dogs-vs-cats\\train\\")
namefiles = []
categories = []
#rename all files under the train subdirectory
for f_name in filenames:
    if "-" not in f_name:
        category = f_name.split('.')[0]  #cat.0.jpg
        f_name_1 = f_name.split('.')[1]+"-"+category+".jpg"  #0-cat.jpg
        os.rename(r'C:\\Users\\xxx\\dogs-vs-cats\\train\\'+f_name, r'C:\\Users\\xxx\\dogs-vs-cats\\train\\'+f_name_1)
        namefiles.append(f_name_1)
    else:
        category = re.split("[-.]",f_name)[1]  #0-cat.jpg
        namefiles.append(f_name)
        
    if category == 'dog':
        categories.append(1)
    else:
        categories.append(0)

df = pd.DataFrame({
    'filename' : namefiles,
    'category' : categories
})

df[1:20]

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers

# pre-trained model 1 -- VGG16, top 3 layers not included, val_accuracy: 0.9010
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128,128,3))

# pre-trained model 2 -- InceptionV3
# from tensorflow.keras.applications.inception_v3 import InceptionV3 #val_accuracy: 0.9495
# base_model = InceptionV3(input_shape = (128, 128, 3), include_top = False, weights = 'imagenet')

# pre-trained model 3 -- ResNet50
# from keras.applications.resnet import ResNet50 # val_accuracy: 0.6821
# base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(128,128,3))

# pre-trained model 4 -- EfficientNetB7
# from tensorflow.keras.applications import EfficientNetB7 #val_accuracy: 0.5016
# base_model = EfficientNetB7(weights='imagenet', include_top=False, input_shape=(128,128,3))

x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
#Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.2)(x)  #added 20220322
x = layers.Dense(128, activation='relu')(x)
#Add a final Fully Connected Sigmoid Layer
x = layers.Dense(2, activation='sigmoid')(x)

model_1 = Model(inputs=base_model.input, outputs=x)

for layer in base_model.layers:
    layer.trainable = False

model_1.compile(optimizer = optimizers.Adam(learning_rate=0.005), loss='categorical_crossentropy', metrics=["accuracy"]) 
model_1.summary()

#Define callbacks and learning rate
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystopping = EarlyStopping(patience = 10)
learning_rate_reduction = ReduceLROnPlateau(monitor = "val_accuracy" , 
                                            verbose = 1, patience =2, 
                                            factor = 0.5, min_lr = 0.00001)

# construct the callback to save only the best model to disk
# use the ModelCheckpoint callback in Keras to save the best version 
# of your model during training
checkpoint_filepath = ".\\checkpoint"
checkpoint = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

callbacks = [earlystopping,learning_rate_reduction,checkpoint]

#adopting StratifiedKFold for learning improvement

df["category"] = df["category"].replace({0:'cat',1:'dog'})
batch_size=32
from sklearn.model_selection import StratifiedKFold
df['Kfold']=-1
df.shape[0]
df=df.sample(frac=1).reset_index(drop=True)
df.shape[0]
y=df['category']

kf=StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for f,(t_,v_) in enumerate(kf.split(X=df,y=y)):
    print("res:"+str(f)+","+str(t_)+","+str(v_)) 
    df.loc[v_,'Kfold']=f
train_df=df[df['Kfold']!=4]
train_df.shape[0]
validate_df=df[df['Kfold']==4]
total_train=train_df.shape[0]
total_validate=validate_df.shape[0]
print (f"train: {total_train}")
print (f"val: {total_validate}")

#Training and validation data generator:


train_datagen = ImageDataGenerator(rotation_range=40,
                                rescale=1./255,
                                shear_range=0.1,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                width_shift_range=0.1,
                                height_shift_range=0.1
                                )
train_generator = train_datagen.flow_from_dataframe(train_df,
                                "C:\\Users\\xxx\\dogs-vs-cats\\train\\",
                                x_col='filename',
                                y_col='category',
                                target_size=image_size,
                                class_mode='categorical',
                                batch_size=batch_size)
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    validate_df, 
    "C:\\Users\\xxx\\dogs-vs-cats\\train\\", 
    x_col='filename',
    y_col='category',
    target_size=image_size,
    class_mode='categorical',
    batch_size=batch_size
)


#Model Training:
epochs=10
history = model_1.fit_generator(
    train_generator, 
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=total_validate//batch_size,
    steps_per_epoch=total_train//batch_size,
    callbacks=callbacks
)

#https://keras.io/api/callbacks/model_checkpoint/
# The model weights (that are considered the best) are loaded into the model.
model_1.load_weights(checkpoint_filepath)

#Save the model:
model_1.save("model_catsVSdogs_10epoch.h5")

#Test data preparation:

test_filenames = os.listdir("C:\\Users\\xxx\\dogs-vs-cats\\test1\\")
test_df = pd.DataFrame({
    'filename': test_filenames
})
nb_samples = test_df.shape[0]



test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = train_datagen.flow_from_dataframe(test_df,
                                "C:\\Users\\xxx\\dogs-vs-cats\\test1\\",
                                x_col='filename',
                                y_col=None,
                                target_size=Image_Size,
                                class_mode=None,
                                batch_size=batch_size)

#Make categorical prediction:

predict = model_1.predict_generator(test_generator, steps=np.ceil(nb_samples/batch_size))

#Convert labels to categories:
#check prediction result
test_df['category'] = np.argmax(predict, axis=1)
label_map = dict((v,k) for k,v in train_generator.class_indices.items())
test_df['category'] = test_df['category'].replace(label_map)
test_df['category'] = test_df['category'].replace({ 'dog': 1, 'cat': 0 })
test_df[0:10]